In [ ]:
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier

from ssri_interactions.io import load_derived_generic
from ssri_interactions.config import Config
from ssri_interactions.decoding.loaders import FSDecodeDataLoader, FSFastDecodeDataLoader
from ssri_interactions.decoding.preprocessors import DecodePreprocessor
from ssri_interactions.decoding.runners import DecodeRunner
from ssri_interactions.decoding.decoders import Decoder
from ssri_interactions.decoding.shuffle import shuffle_X

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
def make_slow_ts_runner(loader):
    preprocessor = DecodePreprocessor(
        thresh_empty=2,
    )
    spikes, states = loader()
    spikes, states = preprocessor(spikes, states)
    estimator = LGBMClassifier(n_estimators=50, max_depth=8, num_leaves=30)
    cv = KFold(n_splits=5, shuffle=True)
    decoder = Decoder(estimator=estimator, cv=cv, shuffler=shuffle_X)
    runner = DecodeRunner(
        loader=loader,
        preprocessor=preprocessor,
        decoder=decoder,
        nboot=75,
    )
    return runner

#### Loaders

In [ ]:
neuron_types = load_derived_generic("neuron_types.csv").query("group in ('CIT', 'SAL')")
sessions = neuron_types.query("experiment_name == 'HAMILTON'").session_name.unique().tolist()
session = sessions[0]
loader_post_shock = FSDecodeDataLoader(session_name=session)
loader_shock_only = FSDecodeDataLoader(session_name=session, t_stop=600)

#### Shock Only

In [ ]:
runner_shock_only = make_slow_ts_runner(loader_shock_only)

pop, unit = runner_shock_only.run_multiple(sessions)
limit = runner_shock_only.run_multiple_limit(sessions=sessions, n_min=1, n_max=15)
dropout = runner_shock_only.run_multiple_dropout(sessions=sessions, neuron_types=("SR", "SIR", "FF"))


dd = Config.derived_data_dir / "decoding"
dd.mkdir(exist_ok=True)
pop.to_csv(dd / "fs_slow - shock only - pop.csv", index=False)
unit.to_csv(dd / "fs_slow - shock only - unit.csv", index=False)
limit.to_csv(dd / "fs_slow - shock only - limit.csv", index=False)
dropout.to_csv(dd / "fs_slow - shock only - dropout.csv", index=False)

#### Post-included

In [ ]:
runner_post_shock = make_slow_ts_runner(loader_post_shock)

pop, unit = runner_post_shock.run_multiple(sessions)
limit = runner_post_shock.run_multiple_limit(sessions=sessions, n_min=1, n_max=15)
dropout = runner_post_shock.run_multiple_dropout(sessions=sessions, neuron_types=("SR", "SIR", "FF"))


dd = Config.derived_data_dir / "decoding"
dd.mkdir(exist_ok=True)
pop.to_csv(dd / "fs_slow - post-included - pop.csv", index=False)
unit.to_csv(dd / "fs_slow - post-included - unit.csv", index=False)
limit.to_csv(dd / "fs_slow - post-included - limit.csv", index=False)
dropout.to_csv(dd / "fs_slow - post-included - dropout.csv", index=False)